In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder()
    .appName("Capítulo 2")
    .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://EM2021002716.bosonit.local:4041
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1617985782383)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@24c13fe9


# Capítulo 2

    a. Descargar el Quijote 
    https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79
    
    Aplicar no solo count (para obtener el número de líneas) y show sino probar 
    distintas sobrecargas del método show (con/sin truncate, indicando/sin indicar 
    num de filas, etc) así como también los métodos, head, take, first (diferencias 
    entre estos 3?) 

In [23]:
var quijote = spark.read.textFile("C:/data/el_quijote.txt")

// Count número de líneas
var lineasQuijote = quijote.count()

// Utilización método show
quijote.show()
quijote.show(10)
quijote.show(5, false) // sin truncar el resultado, muestra 5 líneas, por defecto es true

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPI?TULO 1: Que ...|
|En un lugar de la...|
|Tuvo muchas veces...|
|En resolucio?n, e...|
|historia ma?s cie...|
|Deci?a e?l, que e...|
|En efecto, remata...|
|Imagina?base el p...|
|linaje y patria, ...|
|Limpias, pues, su...|
|Capi?tulo 2: Que ...|
|Hechas, pues, est...|
|Estos pensamiento...|
|Con estos iba ens...|
|Autores hay que d...|
|muertos de hambre...|
+--------------------+
only showing top 20 rows

+--------------------+
|               value|
+--------------------+
|DON QUIJOTE DE LA...|
|Miguel de Cervant...|
|                    |
|       PRIMERA PARTE|
|CAPI?TULO 1: Que ...|
|En un lugar de la...|
|Tuvo muchas veces...|
|En resolucio?n, e...|
|historia ma?s cie...|
|Deci?a e?l, que e...|
+--------------------+
only showing top 10 rows

+-------------------------------------------------------------------------

quijote: org.apache.spark.sql.Dataset[String] = [value: string]
lineasQuijote: Long = 2186
res7: String = DON QUIJOTE DE LA MANCHA


In [33]:
// Métodos head, take y first
quijote.head()  // Sin parámetro devuelve la primera fila

res17: String = DON QUIJOTE DE LA MANCHA


In [32]:
quijote.take(1) // Necesita parámetro y devuelve siempre un Array, aunque el valor sea 1

res16: Array[String] = Array(DON QUIJOTE DE LA MANCHA)


In [28]:
quijote.first() // Devuelve siempre el primer valor, no acepta parámetros

res12: String = DON QUIJOTE DE LA MANCHA


    b. Del ejercicio de M&M aplicar: 
    
    1. Otras operaciones de agregación como el Max con otro tipo de 
    ordenamiento (descendiente). 
    2. hacer un ejercicio como el “where” de CA que aparece en el libro pero 
    indicando más opciones de estados (p.e. NV, TX, CA, CO). 
    3. Hacer un ejercicio donde se calculen en una misma operación el Max, 
    Min, Avg, Count. Revisar el API (documentación) donde encontrarán 
    este ejemplo: 
    ds.agg(max($"age"), avg($"salary")) 
    ds.groupBy().agg(max($"age"), avg($"salary")) 
    NOTA: $ es un alias de col() 
    4. Hacer también ejercicios en SQL creando tmpView 